## Programming Drill 4.1.1
### Write a program that simulates the first quantum system described in this section. The user should be able to specify how many points the particle can occupy (warning: keep the max number low, or you will fairly quickly run out of memory). The user will also specify a ket state vector by assigning its amplitudes. The program, when asked the likelihood of finding the particle at a given point, will perform the calculations described in Example 4.1.1. If the user enters two kets, the system will calculate the probability of transtioning from the first ket to the second, after an observation has been made.

In [9]:
#!/usr/bin/env python3
"""
Programming Drill 4.1.1 (textbook-correct)

- User chooses dimension N.
- User supplies one or two kets (complex amplitude lists of length N).
  * With one ket: program computes norm, per-site probabilities, and optional site query.
  * With two kets: program normalizes both and computes transition probability:
      P = |<phi_norm | psi_norm>|^2
Parsing supports expressions like 1/sqrt(2), sqrt(3), 0.707+0.707j, -2j, etc.
"""

import numpy as np
import math

# ---------------- parsing helpers ----------------
def parse_token_to_complex(tok: str) -> complex:
    """Parse a numeric expression into complex.
       Supports: Python complex literals (1+2j) and simple expressions using sqrt(...).
    """
    s = tok.strip()
    # try Python complex literal first
    try:
        return complex(s)
    except Exception:
        pass
    # replace sqrt with math.sqrt and eval in restricted environment
    safe = s.replace("sqrt", "math.sqrt")
    allowed = {"__builtins__": None, "math": math}
    try:
        val = eval(safe, allowed, {})
        return complex(val)
    except Exception as e:
        raise ValueError(f"Cannot parse '{tok}': {e}")

def read_int(prompt: str) -> int:
    while True:
        try:
            v = int(input(prompt).strip())
            return v
        except Exception:
            print("Please enter a valid integer.")

def read_ket(N: int, prompt: str) -> np.ndarray:
    while True:
        line = input(prompt).strip()
        parts = line.split()
        if len(parts) != N:
            print(f"Expected {N} entries but got {len(parts)}. Enter space-separated amplitudes.")
            continue
        try:
            vals = [parse_token_to_complex(p) for p in parts]
            return np.array(vals, dtype=np.complex128)
        except Exception as e:
            print("Parse error:", e)

# ---------------- quantum helpers ----------------
def norm(vec: np.ndarray) -> float:
    return np.linalg.norm(vec)

def normalize(vec: np.ndarray) -> np.ndarray:
    n = norm(vec)
    if n == 0:
        raise ValueError("Cannot normalize the zero vector.")
    return vec / n

def probabilities(vec: np.ndarray) -> np.ndarray:
    return np.abs(vec)**2

def inner_product(u: np.ndarray, v: np.ndarray) -> complex:
    # <u|v>  (conjugate of u times v)
    return np.vdot(u, v)   # vdot returns conj(u).dot(v)

# ---------------- main ----------------
def main():
    print("Programming Drill 4.1.1 — state vector / transition probability calculator\n")
    N = read_int("Enter number of positions N (dimension): ")
    if N <= 0:
        print("Dimension must be positive.")
        return

    mode = ""
    while mode not in ("1","2"):
        mode = input("Enter '1' to input ONE ket (compute probabilities), or '2' to input TWO kets (compute transition prob): ").strip()

    if mode == "1":
        psi = read_ket(N, f"Enter |psi> as {N} space-separated complex amplitudes: ")
        psi_norm = norm(psi)
        print(f"\n||psi|| = {psi_norm:.12g}")
        probs = probabilities(psi)
        print("\nProbabilities at each position (unnormalized amplitudes):")
        for i, p in enumerate(probs):
            print(f" position {i}: |psi_{i}|^2 = {p:.12g}")
        # Ask optional site query
        q = input("\nEnter an index 0..N-1 to ask probability at that site, or press Enter to skip: ").strip()
        if q != "":
            try:
                i = int(q)
                if not (0 <= i < N):
                    print("Index out of range.")
                else:
                    print(f"Probability at site {i} = {probs[i]:.12g}")
            except Exception:
                print("Invalid index.")
        # also show normalized probabilities if desired
        ans = input("\nNormalize |psi| and show normalized probabilities? (y/[n]) ").strip().lower()
        if ans == "y":
            psi_n = normalize(psi)
            probs_n = probabilities(psi_n)
            print("\nNormalized probabilities (sum should be 1):")
            for i,p in enumerate(probs_n):
                print(f" position {i}: {p:.12g}")
            print(f" Sum = {probs_n.sum():.12g}")

    else:  # mode == "2"
        print("Enter the FIRST ket |psi> (source state):")
        psi = read_ket(N, f"Enter |psi> ({N} amplitudes): ")
        print("Enter the SECOND ket |phi> (target state):")
        phi = read_ket(N, f"Enter |phi> ({N} amplitudes): ")

        print("\nYou entered the following kets:")

        print("\n|psi> =")
        for i, amp in enumerate(psi):
            print(f"  |{i}> : {amp}")

        print("\n|phi> =")
        for i, amp in enumerate(phi):
            print(f"  |{i}> : {amp}")


        # compute norms and normalized kets
        n_psi = norm(psi)
        n_phi = norm(phi)
        print(f"\n||psi|| = {n_psi:.12g},  ||phi|| = {n_phi:.12g}")
        if n_psi == 0 or n_phi == 0:
            print("One of the vectors is zero; cannot compute transition.")
            return
        psi_n = psi / n_psi
        phi_n = phi / n_phi

        ip= inner_product(phi, psi)
        print(f" Inner produt is {ip}")

        # transition probability: |<phi_norm | psi_norm>|^2
        overlap = inner_product(phi_n, psi_n)   # <phi|psi>
        prob = abs(overlap)**2
        print(f"\nOverlap <phi_norm | psi_norm> = {overlap}")
        print(f"Transition probability P(psi -> phi) = |<phi_norm|psi_norm>|^2 = {prob:.12g}")

        # optional: print per-site probabilities of psi and phi
        if input("\nShow normalized per-site probabilities for psi and phi? (y/[n]) ").strip().lower() == "y":
            p_psi = probabilities(psi_n)
            p_phi = probabilities(phi_n)
            print("\nIndex | P_psi (normed) | P_phi (normed)")
            for i in range(N):
                print(f"{i:5d}   {p_psi[i]:18.12g}   {p_phi[i]:18.12g}")
            print(f"\nSum P_psi = {p_psi.sum():.12g}, Sum P_phi = {p_phi.sum():.12g}")

# ---------------- example run helper ----------------
def example_book():
    # Book Example 4.1.1: psi = [-3 - i, -2i, i, 2]
    psi = np.array([-3-1j, 0-2j, 0+1j, 2+0j], dtype=np.complex128)
    print("\nBook Example 4.1.1")
    print("psi = ", psi)
    psi_norm = norm(psi)
    print(f"||psi|| = {psi_norm:.12g}")
    probs = probabilities(psi)
    print("Probabilities (|psi_i|^2):")
    for i,p in enumerate(probs):
        print(f" position {i}: {p:.12g}")
    print(f"Probability at x2 (index 2) = {probs[2]:.12g}")

if __name__ == "__main__":
    main()


Programming Drill 4.1.1 — state vector / transition probability calculator

Enter the FIRST ket |psi> (source state):
Enter the SECOND ket |phi> (target state):

You entered the following kets:

|psi> =
  |0> : (1+0j)
  |1> : -1j

|phi> =
  |0> : 1j
  |1> : (1+0j)

||psi|| = 1.41421356237,  ||phi|| = 1.41421356237
 Inner produt is -2j

Overlap <phi_norm | psi_norm> = -0.9999999999999998j
Transition probability P(psi -> phi) = |<phi_norm|psi_norm>|^2 = 1

Index | P_psi (normed) | P_phi (normed)
    0                  0.5                  0.5
    1                  0.5                  0.5

Sum P_psi = 1, Sum P_phi = 1
